In [28]:
from langchain_core.output_parsers import StrOutputParser
import os

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_core.exceptions import OutputParserException


class Model:
    def __init__(self, model_type):
        self.model = Ollama(model=model_type)
        self.response_schemas = [
            # ResponseSchema(name="explanation", description="explain which part of the old comment should be modified."),
            ResponseSchema(name="newComment", description="the new comment"),
        ]
        self.output_parser = StructuredOutputParser.from_response_schemas(self.response_schemas)
        self.format_instructions = self.output_parser.get_format_instructions()
        self.prompt = PromptTemplate(
            template=
            '''
            Read the following Java method: ```{old_method}```
            Read the following javadoc comment belonging to the method mentioned before: {old_comment}
            Then the Java method was modified to ```{new_method}```
            Please change the comment to fit the new method. 
            The fewer changes, the better. Answer the comment only.
            {format_instructions}''',
            input_variables=["old_method", "old_comment", "new_method"],
            partial_variables={"format_instructions": ''},
        )
        self.chain = self.prompt | self.model | StrOutputParser()

    def resolve(self, old_method, new_method, old_comment):
        while True:
            try:
                return self.chain.invoke({
                    'old_method': old_method,
                    'new_method': new_method,
                    'old_comment': old_comment
                })
            except OutputParserException as e:
                print(e)


## Single test case

In [40]:
import json

myModel = Model('mistral-openorca')
with open('../data/raw/test.jsonl', 'r') as f:
    lines = f.readlines()
    for nr_line in range(0, 10):
        raw_json = lines[nr_line]
        print(f"## Case {nr_line + 1}")
        parsed = json.loads(raw_json)
        _old_method = parsed['src_method']
        _new_method = parsed['dst_method']
        _old_comment = parsed['src_javadoc']
        result = myModel.resolve(_old_method, _new_method, _old_comment)
        print(f"Old     : {_old_comment}")
        print(f"Expected: {parsed['dst_javadoc']}")
        print(f'Actual  : {result.strip()}')
        print()

## Case 1
Old     : Examines the undistorted gray scake input image for squares.
Expected: Examines the undistorted gray scale input image for squares. If p
Actual  : Examines the undistorted gray scale input image for squares and other shapes by detecting contours and finding polygons from the detected contours.

## Case 2
Old     : Save basic clusters.
Expected: Save cluster basic configuration.
Actual  : Save basic cluster configuration.

## Case 3
Old     : Configure a sslConfig for the server using the legacy configuration
Expected: Configure a SSLConfig.Builder for the server using the legacy configuration
Actual  : Configure a SSLConfig.Builder for the server using the legacy configuration

## Case 4
Old     : Wrap an object, if necessary. If the object is null, return the NULL
object. If it is an array or collection, wrap it in a JSONArray. If
it is a map, wrap it in a JSONObject. If it is a standard property
(Double, String, et al) then it is already wrapped. Otherwise, if it


# Multiple test cases

In [11]:
import json

parsed_lines = []
with open('../data/processed/test.jsonl', 'r') as f:
    for line in f:
        parsed = json.loads(line)
        parsed_lines.append(parsed)

result_lines = []
myModel = Model('mistral-openorca')
count = 0
with open('../data/processed/result.jsonl', 'w+') as r:
    for data in parsed_lines:
        _old_method = parsed['src_method']
        _new_method = parsed['dst_method']
        _old_comment = parsed['src_javadoc']
        result = myModel.resolve(_old_method, _new_method, _old_comment)
        r.write(result)
        r.write(os.linesep)
        count += 1
        print(f'Processing... {count}\r')


Processing... 1
Processing... 2
Processing... 3
Processing... 4
Processing... 5
Processing... 6
Processing... 7
Processing... 8
Processing... 9
Processing... 10
Processing... 11
Processing... 12
Processing... 13
Processing... 14
Processing... 15
Processing... 16
Processing... 17
Processing... 18
Processing... 19
Processing... 20
Processing... 21
Processing... 22
Processing... 23
Processing... 24
Processing... 25
Processing... 26
Processing... 27
Processing... 28
Processing... 29
Processing... 30
Processing... 31
Processing... 32
Processing... 33
Processing... 34
Processing... 35
Processing... 36
Processing... 37
Processing... 38
Processing... 39
Processing... 40
Processing... 41
Processing... 42
Processing... 43
Processing... 44
Processing... 45
Processing... 46
Processing... 47
Processing... 48
Processing... 49
Processing... 50
Processing... 51
Processing... 52
Processing... 53
Processing... 54
Processing... 55
Processing... 56
Processing... 57
Processing... 58
Processing... 59
Proces